In [ ]:
!pip install transformers==4.36.0
!pip install peft timm

In [ ]:
!pip install google-search-results

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

In [ ]:
# test.py
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True, torch_dtype=torch.bfloat16)
# For Nvidia GPUs support BF16 (like A100, H100, RTX3090)
# model = model.to(device='cuda', dtype=torch.bfloat16)
# For Nvidia GPUs do NOT support BF16 (like V100, T4, RTX2080)
model = model.to(device='cuda', dtype=torch.float16)
# For Mac with MPS (Apple silicon or AMD GPUs).
# Run with `PYTORCH_ENABLE_MPS_FALLBACK=1 python test.py`
#model = model.to(device='mps', dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True)
model.eval()


In [ ]:
image = Image.open('/content/white_dog.jpg').convert('RGB')#your image path
image

In [ ]:
question = "Give me what is in image in 5 to 7 words"
msgs = [{'role': 'user', 'content': question }]

res, context, _ = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7
)
print(res)

In [ ]:
import spacy
from scipy.spatial.distance import cosine

# Load SpaCy's English model
nlp = spacy.load("en_core_web_md")

# Lists for hardcoded replacements
replaceable_objects = {"same", "similar", "thing", "things"}
replaceable_adjectives = {"same", "similar"}

# Function to extract object-adjective pairs from text
def extract_object_adjective_pairs(text):
    doc = nlp(text)
    pairs = []

    # Extract nouns and their closest preceding adjectives
    for token in doc:
        if token.pos_ == "NOUN":  # Check if the token is a noun (object)
            adj = "none"  # Default adjective if none found
            # Search for a left-side adjective modifying this noun
            for child in token.lefts:
                if child.pos_ == "ADJ":
                    adj = child.text
                    break  # Take the first adjective found
            pairs.append([token.text, adj])
    return pairs

# Function to refine the query
def refine_query(summary_pairs, query_pairs):
    refined_query = ""

    for (s_obj, s_adj), (q_obj, q_adj) in zip(summary_pairs, query_pairs):
        # Replace objects/adjectives if they belong to hardcoded lists
        if q_obj in replaceable_objects:
            q_obj = s_obj
        if q_adj in replaceable_adjectives:
            q_adj = s_adj

        # If both elements are valid (not "none"), check similarity
        if q_obj != "none" and s_obj != "none" and q_adj != "none" and s_adj != "none":
            summary_obj_vector = nlp(s_obj).vector
            query_obj_vector = nlp(q_obj).vector

            # Replace query object if the vectors are far apart
            if cosine(summary_obj_vector, query_obj_vector) > 0.8:
                q_obj = s_obj

        # Construct the refined query part
        refined_query += f"{q_adj} {q_obj} "

    return refined_query.strip()

# Example usage
image_summary = res
# image_summary = "A black candy on a red table"
# user_query = "A similar toffee on a black table"

# image_summary = "A white dog"
user_query = input('Enter Query :')

# Extract object-adjective pairs from the summary and user query
summary_pairs = extract_object_adjective_pairs(image_summary)
query_pairs = extract_object_adjective_pairs(user_query)

# print("Summary Pairs:", summary_pairs)
# print("Query Pairs:", query_pairs)

# Generate the refined query
refined_query = refine_query(summary_pairs, query_pairs)
print("Web search query:", refined_query)

In [ ]:
from serpapi import GoogleSearch

# Replace with your actual API key
api_key = "9b1d8cc71a60e4280387f9b65751e85cc39cb9eb8a1045acf32a91e99afcf11c"

# Define your search parameters
params = {
    "q": f"{refined_query}",  # Query string
    "location": "india",    # Location (optional)
    "hl": "en",                     # Language (optional)
    "gl": "us",                     # Country code (optional)
    "api_key": '9b1d8cc71a60e4280387f9b65751e85cc39cb9eb8a1045acf32a91e99afcf11c'              # Your SerpAPI key
}

# Execute the search
search = GoogleSearch(params)
results = search.get_dict()

# Display the search results
for result in results.get("organic_results", []):
    print(f"Title: {result['title']}")
    print(f"Link: {result['link']}")
    print(f"Snippet: {result.get('snippet', 'No snippet available')}\n")